In [1]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/mental/mental-bert-base-uncased

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/mental/mental-bert-base-uncased)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [2]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="mental/mental-bert-base-uncased")

config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("mental/mental-bert-base-uncased")

In [7]:
!pip install -q transformers datasets accelerate scikit-learn

import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import torch


In [8]:
train_df = pd.read_csv("both_train.csv")
val_df   = pd.read_csv("both_val.csv")
test_df  = pd.read_csv("both_test.csv")

TITLE_COL = "title"
BODY_COL  = "post"
LABEL_COL = "class_id"

# Keep only text + label, drop rows with missing values
train_df = train_df[[TITLE_COL, BODY_COL, LABEL_COL]].dropna()
val_df   = val_df[[TITLE_COL, BODY_COL, LABEL_COL]].dropna()
test_df  = test_df[[TITLE_COL, BODY_COL, LABEL_COL]].dropna()

#combining title and post into one string
train_df["text"] = train_df[TITLE_COL].fillna("") + " " + train_df[BODY_COL].fillna("")
val_df["text"]   = val_df[TITLE_COL].fillna("") + " " + val_df[BODY_COL].fillna("")
test_df["text"]  = test_df[TITLE_COL].fillna("") + " " + test_df[BODY_COL].fillna("")

train_df["labels"] = train_df[LABEL_COL].astype(int)
val_df["labels"]   = val_df[LABEL_COL].astype(int)
test_df["labels"]  = test_df[LABEL_COL].astype(int)

num_labels = train_df["labels"].nunique()
print("num_labels:", num_labels)


num_labels: 6


In [11]:
train_dataset = Dataset.from_pandas(train_df[["text", "labels"]])
val_dataset   = Dataset.from_pandas(val_df[["text", "labels"]])
test_dataset  = Dataset.from_pandas(test_df[["text", "labels"]])

datasets = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset,
})

In [9]:
model_name = "mental/mental-bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,   # you can increase to 256 if you want
    )

tokenized_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/13727 [00:00<?, ? examples/s]

Map:   0%|          | 0/1488 [00:00<?, ? examples/s]

Map:   0%|          | 0/1488 [00:00<?, ? examples/s]

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels,
        preds,
        average="macro"   # or "weighted" if your classes are imbalanced
    )
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "f1": f1,
        "recall": recall,
        "precision": precision,
    }


In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_ds = dataset["train"]
test_ds = dataset["test"]


In [15]:
training_args = TrainingArguments(
    output_dir="./mentalbert_reddit_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-1690069713.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
# Train
trainer.train()

# Validation metrics
val_metrics = trainer.evaluate(tokenized_datasets["validation"])
print("Validation metrics:", val_metrics)

# Test metrics
test_metrics = trainer.evaluate(tokenized_datasets["test"])
print("Test metrics:", test_metrics)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


wandb: Detected [huggingface_hub.inference] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
1,0.690600,0.437922,0.843414,0.845113,0.843414,0.852005
2,0.343400,0.434122,0.858199,0.858634,0.858199,0.860465
3,0.205300,0.490026,0.854167,0.855139,0.854167,0.858480


Validation metrics: {'eval_loss': 0.4341220557689667, 'eval_accuracy': 0.8581989247311828, 'eval_f1': 0.8586341219188153, 'eval_recall': 0.8581989247311829, 'eval_precision': 0.8604653911058877, 'eval_runtime': 9.7105, 'eval_samples_per_second': 153.236, 'eval_steps_per_second': 4.84, 'epoch': 3.0}
Test metrics: {'eval_loss': 0.4022432863712311, 'eval_accuracy': 0.8689516129032258, 'eval_f1': 0.8686552608546289, 'eval_recall': 0.8689516129032259, 'eval_precision': 0.8703438311676361, 'eval_runtime': 9.8441, 'eval_samples_per_second': 151.157, 'eval_steps_per_second': 4.774, 'epoch': 3.0}
